In [ ]:
import pandas as pd
import re
from statistics import mean
from collections import Counter
from utils import TYPES
# TYPES=['[I]ntro/[E]xtra-vert', 'I[n]tuition/[S]ensing', '[T]hink/Feel', '[J]udge/[P]erceive']
DOMAIN_PATTERN='^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/\n]+)'
URL_PATTERN='http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

In [ ]:
data = pd.read_csv("mbti_data.csv")
data['posts']= data['posts'].apply(lambda x : x[1:-1])
data.head()

In [ ]:
len(data['posts'][0].split("|||"))
lens = [len(data['posts'][i].split(r"|||")) for i in range(len(data))]
cnt = Counter(lens)
print(mean(lens))
print(cnt)

In [ ]:
data['no_posts'] = data['posts'].apply(lambda x: len(x.split(r"|||")))
data = data[ data['no_posts'] <= 50]
data = data[ data['no_posts'] >= 30]

In [ ]:
data['no_posts'].hist()

In [ ]:
data['type'].value_counts()

In [ ]:
split_df = data['type'].str.split('',expand=True).drop([0, 5], axis=1)
split_df.columns = TYPES
split_df.head()

In [ ]:
data = pd.concat([split_df, data], axis=1)

In [ ]:
data.reset_index(inplace=True)
data.rename(columns = {'index':'user_id'}, inplace = True)

In [ ]:
data.head()

In [ ]:
for type_ in TYPES:
    print(data[type_].value_counts())

In [ ]:
data.head()

In [ ]:
data_split = data.set_index(['user_id',*TYPES,'type','no_posts']).apply(lambda x: x.str.split(r'\|\|\|').explode()).reset_index()

In [ ]:
data_split

In [ ]:
for type_ in TYPES:
    print(data_split[type_].value_counts())

## Text preprocessing

In [ ]:
def text_preprocessing(text):
    # Remove unhandled unicode chars (#1231)
    text = re.sub(r"( #\d*;)",
              lambda m: html.unescape('&' +m.group(1)[1:]),
              text.rstrip())

    # Replace '&amp;' with '&'
    text = re.sub(r'&amp;', '&', text)



    # Remove trailing whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    text = " ".join(text.split())
    text = re.sub(URL_PATTERN,"<URL>", text)
    
    return text

In [ ]:
data_split['posts'] = data_split['posts'].apply(lambda x: text_preprocessing(x))

In [ ]:
data_split.dropna(inplace=True)

In [ ]:
# data_split.drop(["no_posts"],axis=1).to_csv("mbti_processed.csv", index=False)

In [ ]:
# data_split = pd.read_csv("mbti_processed.csv")

In [ ]:
# data_split.head()

In [ ]:
# data_split['posts'].isna().sum()

In [ ]:
# data_split.isna().sum().sum()